# PlatyHash
## Hash laboratorio 3 Cripto

El proposito de este hash es lograr hashear un texto

In [20]:
import sys
import io
import codecs
from google.colab import files
import time
import hashlib

In [60]:
import string
alph = list(string.ascii_letters + string.digits + string.punctuation)
print(len(alph))

94


Funcion de calculo de entropia, la entropia siempre es 94

In [57]:
import math

def Entropia(n,base):
  if(base==0):
    base_hash = 94
  else:
    base_hash = base
  entropia = round(n*math.log2(base_hash),3)
  return str(entropia)
  

In [23]:
def SumDig(x):
  sumDigit = 0
  while x != 0:
      extNum = x % 10
      x //= 10
      sumDigit += extNum
  return sumDigit

En este apartado se busca las letras de una palabra, separando estas de los numeros.

In [25]:
def letras(palabra):
  vocal = ""
  for i in range(len(palabra)):
    try:
      n = 1 + int(palabra[i])
    except:
      vocal = vocal + palabra[i]
  return vocal

La primera funcion entrega el codigo hexadecimal y el unicode de la palabra.

La segunda funcion busca las letras y los numeros de un hexadecimal y los separa.

La tercera funcion crea un padding con la suma de los numeros hexadecimales, si esta es menor a 10, se multiplicara por dos hasta que este cumpla la condicion anterior.

In [54]:
def realHash(palabra):
  hexad = ""
  uni = ""
  #print(len(palabra))
  for i in range(len(palabra)):
    uni = uni + str(ord(palabra[i]))
    hexad = hexad + str(palabra[i].encode('utf-8').hex())
  return [hexad, uni]
  
def realHash2(hexa, uni):
  vocal=""
  pad = 0
  for i in range(len(hexa)):
    try:
      pad = pad + int(hexa[i]) + i
    except:
      vocal = vocal + hexa[i]

  return [vocal,pad]

def Padding(pad):
  while(int(pad)<10):
    pad = int(pad)*2
  return str(str(pad).encode('utf-8').hex())

Esta funcion intercala dos palabras y forma una nueva palabra combinando las dos palabras.

In [7]:
def Intercalar2(palabra1, palabra2):
  j=0
  k=0
  if(len(palabra1)>=len(palabra2)):
    uni = palabra1
    vocal = palabra2
  else:
    uni = palabra2
    vocal = palabra1
  #print(vocal, "->",uni )
  for i in range(1,len(uni)):
    uni = uni[0:k] + vocal[j] + uni[k:len(uni)]
    j = j+1
    k = k+1
  return uni

Esta funcion evalua si existen letras en el hexadecimal, en caso de que no, se dan unas letras estandar puestas arbitrariamente. Despues se van intercalando estas letras en el unicode de tal forma que el unicode quede intercalando numeros y letras.

In [8]:
def PalHash(vocal, uni, palabra):
  letrext = ['v','e','#']
  if(not len(vocal) and letras(palabra)):
    vocal = letras(palabra)
  u = 0
  while(len(vocal)<3):
    vocal = vocal + letrext[u]
    u=u+1
  j=0
  k=1
  for i in range(1,len(uni)):
    try:
      uni = uni[0:k] + vocal[j] + uni[k:len(uni)]
      j = j+1
      k = k+2
    except:
      j = 0
      uni = uni[0:k] + vocal[j] + uni[k:len(uni)]
      j = j+1
      k = k+2
  if(len(uni)%2 is not 0):
    uni = uni + "&"
  return uni

Este es el ultimo paso, evalua el tamaño de lo que se lleva de hash y si es que es muy pequeño, se crea tantas veces un padding para extender el hash.

In [9]:
def PadDef(palabra):
  npala=palabra
  while(len(npala)<=55):
    coHex = realHash(npala)[0]
    coUni = realHash(npala)[1]
    padd = realHash2(coHex,coUni)[1]
    npala = npala + str(padd)
    #npala = Intercalar2(npala[0:int(len(npala)/2)],npala[int(len(npala)/2):len(npala)])
  return npala[0:55]

En aplicar, se usan todas las funciones descritas anteiormente en orden y retorna el ultimo hash que entregue la funcion PadDef

In [14]:
def Aplicar(palabra):
  i = 0 
  coHex = realHash(palabra)[0]
  coUni = realHash(palabra)[1]

  while(len(palabra) < 5):
    palabra = palabra + alph[int(coUni[i])]
    i = i+1

  letra = realHash2(coHex,coUni)[0]
  padd = realHash2(coHex,coUni)[1]
  Apad = Padding(padd)
  princ = PalHash(letra,coUni,palabra)
  segund = Intercalar2(princ[0:int(len(princ)/2)],princ[int(len(princ)/2):len(princ)])
  mpad = segund + Apad
  #tercer = Intercalar2(mpad[0:int(len(mpad)/2)],mpad[int(len(mpad)/2):len(mpad)])

  return PadDef(mpad)

Aquí se puede encontrar un ejemplo del uso de hash

In [64]:
print(Aplicar("hola"))
print(Aplicar("Hola"))

v1f0c8v9f71f0c4v1f1c13533143118802401297636324342511559
1v0f8c9v77f2c1v1f1c353112281649214126853306398247325540


Esta funcion permite subir archivos y que se logre cifrar un archivo.

In [65]:
def SubirArchivo():
  uploaded = files.upload()
  filename = next(iter(uploaded))
  print("Nombre del Archivo: ",filename)
  filePath = "./"+filename
  file1 = open(filePath, 'r')
  count = 0
    
  filePath = "./"+filename

  with open(filePath, 'rb') as file:
    fi  = io.FileIO(file.fileno())
    fb = io.BufferedReader(fi)
    result = codecs.encode(fb.read(),'hex_codec')
    result = result.decode('ascii')
  print("El hash del archivo es: ",Aplicar(result))

Esta funcion calcula el tiempo de hash

In [50]:
def Tiempo(palabra):
  inicio=time.time()
  hash = Aplicar(palabra)
  fin=time.time()
  tiempoHash=fin-inicio
  return [hash,tiempoHash]

Esta funcion permite subir un archivo y que lea todas las entradas de texto de cada linea y las hashea, imprime el hash y el tiempo que se tardo.

In [32]:
def Subir():
    up = files.upload()
    filename = next(iter(up))
    print("Nombre: ", filename)
    with open('Pass50.txt', newline='') as File:  
      reader = File.readlines()
      for row in reader:
        row = row.split('\r')[0]
        valor = Tiempo(row)
        print(str(row)," -> ",valor[0],' -> ',valor[1])

Seleccion multiple para saber que se quiere aplicar

In [66]:
from google.colab import files

n = 0
while(n is not 4):
  print("\nSeleccione un de las siguientes Opciones: \n")
  print("1.- Realizar Hash a un String")
  print("2.- Realizar Hash a varios String dentro de un Archivo de Texto")
  print("3.- Realizar Hash a un Archivo")
  print("4.- Salir")

  n = int(input("\nIngrese el numero: "))
  while(n<1 or n >4):
    print("\nPor favor seleccione una de las opciones dadas anteriormente \n")
    n = int(input("\nIngrese el numero: "))
  if(n == 1):
    text = input("Ingrese el String: ")
    print("Texto ingresado: ",text)
    h = Aplicar(text)
    print('\nHash: ',h)
    print("Entropia del Hash: ",Entropia(len(h),0))
  elif(n == 2):
    print("\nSuba el archivo de Texto: ")
    Subir()
  elif(n == 3):
    print("\nSuba el archivo: ")
    SubirArchivo()


Seleccione un de las siguientes Opciones: 

1.- Realizar Hash a un String
2.- Realizar Hash a varios String dentro de un Archivo de Texto
3.- Realizar Hash a un Archivo
4.- Salir

Ingrese el numero: 1
Ingrese el String: hola
Texto ingresado:  hola

Hash:  v1f0c8v9f71f0c4v1f1c13533143118802401297636324342511559
Entropia del Hash:  360.502

Seleccione un de las siguientes Opciones: 

1.- Realizar Hash a un String
2.- Realizar Hash a varios String dentro de un Archivo de Texto
3.- Realizar Hash a un Archivo
4.- Salir

Ingrese el numero: 2

Suba el archivo de Texto: 


Saving Pass50.txt to Pass50 (12).txt
Nombre:  Pass50.txt
123456  ->  5v2e5#3v5e44v9e5#0v5e1#31303518812403298036314340511159  ->  0.0011255741119384766
12345  ->  1#5v2e5#34v9e5#0v5e373512211635212226653284396547165526  ->  0.0012311935424804688
123456789  ->  3v5e4#5v5e5#6v5e74v9e5#0v5e1#5v2e5#32323536614373515059  ->  0.0005645751953125
password  ->  e1f9v1e1f1v1e1f4v1e0f01f1v2e9f7v1e1f5v1e1f5v13138325577  ->  0.0003211498260498047
iloveyou  ->  1c0f1f1c2f1f1c1f1f1c1f71c0f5f1c0f8f1c1f1f1c1f8f31383459  ->  0.0001804828643798828
princess  ->  e9e9v1e0e1v1e1e5v1e1e51e1v2e1e1v4e1e0v5e1e1v03139305565  ->  0.0003294944763183594
1234567  ->  2e5#3v5e4#5v54v9e5#0v5e1#5v3134302406298636434355513259  ->  0.0005838871002197266
rockyou  ->  7b1f2f1b1f1f1b1f1f71f1b4f1f1b1f9f9b1f0f3133344355513259  ->  0.00036263465881347656
12345678  ->  5#3v5e4#5v5e5#64v9e5#0v5e1#5v2e313830299936604371514659  ->  0.0005176067352294922
abc123  ->  4a9b5c0a5b19a7b9c8a9b9c31303518842409299236464361513559  ->  0.000

Saving Pass50.txt to Pass50 (13).txt
Nombre del Archivo:  Pass50.txt
El hash del archivo es:  d5a4d5a7c5f4d1a0d1a5d4c5c3d4a8d1a0e0e4d8a9f7f5c4d5a1f5d

Seleccione un de las siguientes Opciones: 

1.- Realizar Hash a un String
2.- Realizar Hash a varios String dentro de un Archivo de Texto
3.- Realizar Hash a un Archivo
4.- Salir

Ingrese el numero: 7

Por favor seleccione una de las opciones dadas anteriormente 


Ingrese el numero: 0

Por favor seleccione una de las opciones dadas anteriormente 


Ingrese el numero: 4


Comparación hash

Funciones que definene otros algoritmos de hash y que calculan sus tiempos para despues aplicar un print comparativo con la entropia.

In [67]:
def Md5(palabra):
  sb=palabra.encode("utf-8")
  inicio=time.time()
  md5=hashlib.md5(sb).hexdigest()
  fin=time.time()
  return fin-inicio

def Sha1(palabra):
  sb=palabra.encode("utf-8")
  inicio=time.time()
  sha1=hashlib.sha1(sb).hexdigest()
  fin=time.time()
  return fin-inicio

def Sha256(palabra):
  sb=palabra.encode("utf-8")
  inicio=time.time()
  sha256=hashlib.sha256(sb).hexdigest()
  fin=time.time()
  return fin-inicio

def RealCompa(palabra):
  print(palabra," ",Entropia(len(palabra),0)," ",Tiempo(palabra)[1]," ",Md5(palabra)," ",Sha1(palabra)," ",Sha256(palabra))


up = files.upload()
filename = next(iter(up))
print("Nombre: ", filename)
with open(filename, newline='') as File:  
  reader = File.readlines()
  for row in reader:
    RealCompa(row)



Saving Pass50.txt to Pass50 (14).txt
Nombre:  Pass50.txt
123456
   52.437   0.000553131103515625   1.1205673217773438e-05   3.0994415283203125e-06   3.361701965332031e-05
12345
   45.882   0.0007066726684570312   1.0728836059570312e-05   3.5762786865234375e-06   3.5762786865234375e-06
123456789
   72.1   0.00030422210693359375   9.059906005859375e-06   3.0994415283203125e-06   2.86102294921875e-06
password
   65.546   0.00010919570922851562   2.6226043701171875e-06   1.6689300537109375e-06   1.1920928955078125e-06
iloveyou
   65.546   7.62939453125e-05   1.430511474609375e-06   1.430511474609375e-06   1.430511474609375e-06
princess
   65.546   6.842613220214844e-05   1.1920928955078125e-06   1.1920928955078125e-06   1.1920928955078125e-06
1234567
   58.991   0.0006096363067626953   4.0531158447265625e-06   1.6689300537109375e-06   1.9073486328125e-06
rockyou
   58.991   0.00020432472229003906   1.2159347534179688e-05   3.814697265625e-06   3.337860107421875e-06
12345678
   65.546   0.0